In [34]:
from getDataFunctions import getStareData, getRFMIDData, getODIRData
IMAGE_SIZE = (256,256)

In [35]:
#Get All Data
stare_AMD_w_others_data, stare_normal_data = getStareData(IMAGE_SIZE)
RFMiD_Pure_AMD_data, RFMiD_AMD_w_others_data, RFMiD_normal_data = getRFMIDData(IMAGE_SIZE)
ODIR_Pure_AMD_data, ODIR_AMD_w_others_data, ODIR_normal_data = getODIRData(IMAGE_SIZE)


In [36]:
print(ODIR_Pure_AMD_data.shape)

(198, 256, 256, 1)


## U-NET predict Data

In [37]:
from keras_unet.models import custom_unet

input_shape = stare_AMD_w_others_data[0].shape

model = custom_unet(
    input_shape,
    filters=32,
    use_batch_norm=True,
    dropout=0.3,
    dropout_change_per_layer=0.0,
    num_layers=4
)


In [38]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_57 (Conv2D)             (None, 256, 256, 32  288         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_54 (BatchN  (None, 256, 256, 32  128        ['conv2d_57[0][0]']              
 ormalization)                  )                                                           

In [39]:
from keras.optimizers import Adam, SGD
from keras_unet.metrics import iou, iou_thresholded
from tensorflow.keras.metrics import Recall, Precision
from metrics import dice_loss, dice_coef, iou
from tensorflow import keras

loss = keras.losses.BinaryCrossentropy()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=loss,
    metrics=["accuracy", dice_coef, iou, Recall(), Precision()],
)

In [40]:
##load model
model_filename = 'segm_model_v3.h5'

if(os.path.exists(model_filename)):
    model.load_weights(model_filename)


In [41]:
import tensorflow as tf

##predict function
def predict(data):
    with tf.device("CPU"): 
        result = model.predict(data)
        
    return result

## Prediction

In [42]:
stare_AMD_w_others_pred = predict(stare_AMD_w_others_data)
#stare_normal_pred = predict(stare_normal_data)
#RFMiD_Pure_AMD_pred = predict(RFMiD_Pure_AMD_data)
#RFMiD_AMD_w_others_pred = predict(RFMiD_AMD_w_others_data)
#RFMiD_normal_pred = predict(RFMiD_normal_data)
#ODIR_Pure_AMD_pred = predict(ODIR_Pure_AMD_data)
#ODIR_AMD_w_others_pred = predict(ODIR_AMD_w_others_data)
#ODIR_normal_pred = predict(ODIR_normal_data)

2/2 [==============================] - 6s 2s/step
